# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - Some of the options require a Mapbox API token:
    - However, we will be using the options that DO NOT require a token.
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [ ]:
## Plotly is not included in your dojo-env
!pip install plotly

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [24]:
## Load in csv.gz
df = pd.read_csv('Solution/Data/Fresno-wings.csv.gz')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,mlOelmCRqLOkWlLcAIZ-BA,swamp-chicken-missouri-city,Swamp Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/iV2Jhj...,False,https://www.yelp.com/biz/swamp-chicken-missour...,23,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.0,"{'latitude': 29.545382778092428, 'longitude': ...",[],"{'address1': '8035 Hwy 6', 'address2': '', 'ad...",1.281970e+10,(281) 969-8945,7523.026816,NaN
1,zcc818cmdAJ6JDiS8ODXAg,big-city-wings-pearland,Big City Wings,https://s3-media2.fl.yelpcdn.com/bphoto/LN4X0h...,False,https://www.yelp.com/biz/big-city-wings-pearla...,286,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"{'latitude': 29.557198, 'longitude': -95.371067}","['pickup', 'delivery']","{'address1': '9721 Broadway St', 'address2': N...",1.346755e+10,(346) 754-5980,9797.744647,$$
2,kBPaHWjREZNgr35J5l3_jg,wingstop-arcola,Wingstop,https://s3-media1.fl.yelpcdn.com/bphoto/2X_qSY...,False,https://www.yelp.com/biz/wingstop-arcola?adjus...,38,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",2.0,"{'latitude': 29.50753417185821, 'longitude': -...",[],"{'address1': '5243 Fm 521 Rd', 'address2': 'St...",1.281432e+10,(281) 431-9464,3479.181431,NaN
3,b4a8eGt5rBr8ubKGXO8wYA,take-the-wheel-mobile-fresno,Take The Wheel Mobile,https://s3-media2.fl.yelpcdn.com/bphoto/WWd4yD...,False,https://www.yelp.com/biz/take-the-wheel-mobile...,59,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"{'latitude': 29.523673, 'longitude': -95.45414}","['pickup', 'delivery']","{'address1': '4233 Fm 521 Rd', 'address2': Non...",1.832334e+10,(832) 334-2181,2171.094342,$
4,tsLFO0SuZPjqCtGcDNnWig,bar-kada-missouri-city,Bar Kada,https://s3-media3.fl.yelpcdn.com/bphoto/WEs1ij...,False,https://www.yelp.com/biz/bar-kada-missouri-cit...,49,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",3.5,"{'latitude': 29.537163, 'longitude': -95.533925}",[],"{'address1': '9009 Sienna Crossing Dr', 'addre...",1.713228e+10,(713) 227-5232,6197.980513,NaN


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- 2. We also want to be able to show the restaurants:
    - name
    - price
    - type of transactions (pickup/delivery)
    - address

### Separating Latitude and Longitude

In [25]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 29.545382778092428, 'longitude': ..."
1,"{'latitude': 29.557198, 'longitude': -95.371067}"
2,"{'latitude': 29.50753417185821, 'longitude': -..."
3,"{'latitude': 29.523673, 'longitude': -95.45414}"
4,"{'latitude': 29.537163, 'longitude': -95.533925}"
...,...
196,"{'latitude': 29.580023022486706, 'longitude': ..."
197,"{'latitude': 29.55656, 'longitude': -95.38052}"
198,"{'latitude': 29.571845, 'longitude': -95.564201}"
199,"{'latitude': 29.551846, 'longitude': -95.385833}"


- Why didn't that work???

In [ ]:
## slice out a single test coordinate
test_coord = None
test_coord

- Its not a dictionary anymore!!! What??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our `string dictionary` into an `actual dictionary`. 

In [ ]:
## Use json.loads on the test coordinate


### JSON requires double quotes!
We got a `JSON Decode Error` because JSON was `expecting double quotes` inside
of the dictionary

### We are now going to use the .replace( ) function to replace single ' with double "

In [5]:
## replace single ' with double " 
df['coordinates'] = df['coordinates'].str.replace("'", '"')
df['coordinates'][0]

'{"latitude": 29.545382778092428, "longitude": -95.54706904949656}'

In [ ]:
## Use json.loads on the test coordinate, again


In [ ]:
# viewing type after using json.loads


### Now, how can we apply this same process to the entire column??

In [26]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'", '"')
df['coordinates'][0]
## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)

In [27]:
## slice out a single test coordinate
print(type(df['coordinates'][0]))
df['coordinates'][0]

<class 'dict'>


{'latitude': 29.545382778092428, 'longitude': -95.54706904949656}

### Using .apply with pd.Series to convert a dictionary column into multiple columns
This is the process of unpacking the dictionary to columns

In [28]:
## use .apply pd.Series to convert a dict to columns
df_coordinate = df['coordinates'].apply(pd.Series)
df_coordinate

,latitude,longitude
0,29.545383,-95.547069
1,29.557198,-95.371067
2,29.507534,-95.459899
3,29.523673,-95.454140
4,29.537163,-95.533925
...,...,...
196,29.580023,-95.394722
197,29.556560,-95.380520
198,29.571845,-95.564201
199,29.551846,-95.385833


In [29]:
## Concatenate the 2 new columns and drop the original.
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,mlOelmCRqLOkWlLcAIZ-BA,swamp-chicken-missouri-city,Swamp Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/iV2Jhj...,False,https://www.yelp.com/biz/swamp-chicken-missour...,23,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.0,"{'latitude': 29.545382778092428, 'longitude': ...",[],"{'address1': '8035 Hwy 6', 'address2': '', 'ad...",1.281970e+10,(281) 969-8945,7523.026816,NaN
1,zcc818cmdAJ6JDiS8ODXAg,big-city-wings-pearland,Big City Wings,https://s3-media2.fl.yelpcdn.com/bphoto/LN4X0h...,False,https://www.yelp.com/biz/big-city-wings-pearla...,286,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"{'latitude': 29.557198, 'longitude': -95.371067}","['pickup', 'delivery']","{'address1': '9721 Broadway St', 'address2': N...",1.346755e+10,(346) 754-5980,9797.744647,$$
2,kBPaHWjREZNgr35J5l3_jg,wingstop-arcola,Wingstop,https://s3-media1.fl.yelpcdn.com/bphoto/2X_qSY...,False,https://www.yelp.com/biz/wingstop-arcola?adjus...,38,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",2.0,"{'latitude': 29.50753417185821, 'longitude': -...",[],"{'address1': '5243 Fm 521 Rd', 'address2': 'St...",1.281432e+10,(281) 431-9464,3479.181431,NaN
3,b4a8eGt5rBr8ubKGXO8wYA,take-the-wheel-mobile-fresno,Take The Wheel Mobile,https://s3-media2.fl.yelpcdn.com/bphoto/WWd4yD...,False,https://www.yelp.com/biz/take-the-wheel-mobile...,59,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"{'latitude': 29.523673, 'longitude': -95.45414}","['pickup', 'delivery']","{'address1': '4233 Fm 521 Rd', 'address2': Non...",1.832334e+10,(832) 334-2181,2171.094342,$
4,tsLFO0SuZPjqCtGcDNnWig,bar-kada-missouri-city,Bar Kada,https://s3-media3.fl.yelpcdn.com/bphoto/WEs1ij...,False,https://www.yelp.com/biz/bar-kada-missouri-cit...,49,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",3.5,"{'latitude': 29.537163, 'longitude': -95.533925}",[],"{'address1': '9009 Sienna Crossing Dr', 'addre...",1.713228e+10,(713) 227-5232,6197.980513,NaN


In [30]:
df = pd.concat((df, df_coordinate), axis=1)
df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,mlOelmCRqLOkWlLcAIZ-BA,swamp-chicken-missouri-city,Swamp Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/iV2Jhj...,False,https://www.yelp.com/biz/swamp-chicken-missour...,23,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.0,"{'latitude': 29.545382778092428, 'longitude': ...",[],"{'address1': '8035 Hwy 6', 'address2': '', 'ad...",1.281970e+10,(281) 969-8945,7523.026816,NaN,29.545383,-95.547069
1,zcc818cmdAJ6JDiS8ODXAg,big-city-wings-pearland,Big City Wings,https://s3-media2.fl.yelpcdn.com/bphoto/LN4X0h...,False,https://www.yelp.com/biz/big-city-wings-pearla...,286,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"{'latitude': 29.557198, 'longitude': -95.371067}","['pickup', 'delivery']","{'address1': '9721 Broadway St', 'address2': N...",1.346755e+10,(346) 754-5980,9797.744647,$$,29.557198,-95.371067


In [31]:
df.drop(columns='coordinates', inplace=True)
df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,mlOelmCRqLOkWlLcAIZ-BA,swamp-chicken-missouri-city,Swamp Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/iV2Jhj...,False,https://www.yelp.com/biz/swamp-chicken-missour...,23,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.0,[],"{'address1': '8035 Hwy 6', 'address2': '', 'ad...",1.281970e+10,(281) 969-8945,7523.026816,NaN,29.545383,-95.547069
1,zcc818cmdAJ6JDiS8ODXAg,big-city-wings-pearland,Big City Wings,https://s3-media2.fl.yelpcdn.com/bphoto/LN4X0h...,False,https://www.yelp.com/biz/big-city-wings-pearla...,286,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"['pickup', 'delivery']","{'address1': '9721 Broadway St', 'address2': N...",1.346755e+10,(346) 754-5980,9797.744647,$$,29.557198,-95.371067


### Activity -Padlet : Unpacking dictionary

- https://padlet.com/swhaley9/unpacking-dictionaries-n19x0tiyn980jr0o

## Creating a Simple Map

- Mapbox API: https://www.mapbox.com/
- Mapbox API Documentation: https://docs.mapbox.com/api/overview/

- Use the plotly express `scatter_mapbox` function

In [14]:
## use scatter_mapbox for map
px.scatter_mapbox(df, lat='latitude', lon='longitude', mapbox_style='open-street-map',
                 hover_name='name', hover_data=['price', 'rating', 'transactions', 'location'])

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - rating
    - transaction type (delivery/takeout)
    - address
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [ ]:
## add hover_name (name) and hover_data for price,rating,location


### Fixing the Location Column

In [ ]:
## slice out a test address


> Also a string-dictionary...

In [32]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,mlOelmCRqLOkWlLcAIZ-BA,swamp-chicken-missouri-city,Swamp Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/iV2Jhj...,False,https://www.yelp.com/biz/swamp-chicken-missour...,23,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.0,[],"{""address1"": ""8035 Hwy 6"", ""address2"": """", ""ad...",1.281970e+10,(281) 969-8945,7523.026816,NaN,29.545383,-95.547069
1,zcc818cmdAJ6JDiS8ODXAg,big-city-wings-pearland,Big City Wings,https://s3-media2.fl.yelpcdn.com/bphoto/LN4X0h...,False,https://www.yelp.com/biz/big-city-wings-pearla...,286,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"['pickup', 'delivery']","{""address1"": ""9721 Broadway St"", ""address2"": N...",1.346755e+10,(346) 754-5980,9797.744647,$$,29.557198,-95.371067
2,kBPaHWjREZNgr35J5l3_jg,wingstop-arcola,Wingstop,https://s3-media1.fl.yelpcdn.com/bphoto/2X_qSY...,False,https://www.yelp.com/biz/wingstop-arcola?adjus...,38,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",2.0,[],"{""address1"": ""5243 Fm 521 Rd"", ""address2"": ""St...",1.281432e+10,(281) 431-9464,3479.181431,NaN,29.507534,-95.459899
3,b4a8eGt5rBr8ubKGXO8wYA,take-the-wheel-mobile-fresno,Take The Wheel Mobile,https://s3-media2.fl.yelpcdn.com/bphoto/WWd4yD...,False,https://www.yelp.com/biz/take-the-wheel-mobile...,59,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"['pickup', 'delivery']","{""address1"": ""4233 Fm 521 Rd"", ""address2"": Non...",1.832334e+10,(832) 334-2181,2171.094342,$,29.523673,-95.454140
4,tsLFO0SuZPjqCtGcDNnWig,bar-kada-missouri-city,Bar Kada,https://s3-media3.fl.yelpcdn.com/bphoto/WEs1ij...,False,https://www.yelp.com/biz/bar-kada-missouri-cit...,49,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",3.5,[],"{""address1"": ""9009 Sienna Crossing Dr"", ""addre...",1.713228e+10,(713) 227-5232,6197.980513,NaN,29.537163,-95.533925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Pql6fHY89HmU2xYit08FIw,sonic-drive-in-pearland-7,Sonic Drive-In,https://s3-media4.fl.yelpcdn.com/bphoto/3sL9Hk...,False,https://www.yelp.com/biz/sonic-drive-in-pearla...,79,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"['delivery', 'pickup']","{""address1"": ""11309 Shadow Creek Pkwy"", ""addre...",1.713340e+10,(713) 340-0806,8660.337247,$,29.580023,-95.394722
197,bAgiCCzQUO4QyD26sX4YUw,silverlake-hunan-restaurant-pearland-2,Silverlake Hunan Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/o1HT5H...,False,https://www.yelp.com/biz/silverlake-hunan-rest...,100,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.0,['delivery'],"{""address1"": ""10223 Broadway St"", ""address2"": ...",1.713437e+10,(713) 436-9988,8971.510555,$,29.556560,-95.380520
198,xPUTiVph-0PTYL-NAxAvGA,old-hickory-inn-missouri-city,Old Hickory Inn,https://s3-media1.fl.yelpcdn.com/bphoto/rVMKy_...,False,https://www.yelp.com/biz/old-hickory-inn-misso...,82,"[{'alias': 'bbq', 'title': 'Barbeque'}]",3.5,"['delivery', 'pickup']","{""address1"": ""3334 Fm 1092 Rd"", ""address2"": ""S...",1.281500e+10,(281) 499-8904,9891.253375,$$,29.571845,-95.564201
199,D7bjSkrIekLMAv-PB83bGQ,chuck-e-cheese-pearland-2,Chuck E. Cheese,https://s3-media2.fl.yelpcdn.com/bphoto/P_Evbf...,False,https://www.yelp.com/biz/chuck-e-cheese-pearla...,52,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"['delivery', 'pickup']","{""address1"": ""3141 Silverlake Village Dr"", ""ad...",1.713340e+10,(713) 340-1530,8277.198159,$,29.551846,-95.385833


In [ ]:
## apply json.loads


# > Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [33]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

'{"address1": "8035 Hwy 6", "address2": "", "address3": None, "city": "Missouri City", "zip_code": "77459", "country": "US", "state": "TX", "display_address": ["8035 Hwy 6", "Missouri City, TX 77459"]}'

In [34]:
## write a function to just run json.loads on the address
def fix_address(test_addr):
    try:
        return json.loads(test_addr)
    except:
        return 'Error'

In [35]:
## test applying our function
df['location'].apply(fix_address)

0                                                  Error
1                                                  Error
2                                                  Error
3                                                  Error
4                                                  Error
                             ...                        
196                                                Error
197                                                Error
198    {'address1': '3334 Fm 1092 Rd', 'address2': 'S...
199    {'address1': '3141 Silverlake Village Dr', 'ad...
200    {'address1': '6124 Highway 6', 'address2': '',...
Name: location, Length: 201, dtype: object

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [36]:
### save a new display_location column using our function
df['display_location'] = df['location'].apply(fix_address)

In [19]:
## filter for businesses with display_location == "ERROR"
errors = df[df['display_location'] == 'Error']
errors

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location
0,mlOelmCRqLOkWlLcAIZ-BA,swamp-chicken-missouri-city,Swamp Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/iV2Jhj...,False,https://www.yelp.com/biz/swamp-chicken-missour...,23,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.0,[],"{'address1': '8035 Hwy 6', 'address2': '', 'ad...",1.281970e+10,(281) 969-8945,7523.026816,NaN,29.545383,-95.547069,Error
1,zcc818cmdAJ6JDiS8ODXAg,big-city-wings-pearland,Big City Wings,https://s3-media2.fl.yelpcdn.com/bphoto/LN4X0h...,False,https://www.yelp.com/biz/big-city-wings-pearla...,286,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"['pickup', 'delivery']","{'address1': '9721 Broadway St', 'address2': N...",1.346755e+10,(346) 754-5980,9797.744647,$$,29.557198,-95.371067,Error
2,kBPaHWjREZNgr35J5l3_jg,wingstop-arcola,Wingstop,https://s3-media1.fl.yelpcdn.com/bphoto/2X_qSY...,False,https://www.yelp.com/biz/wingstop-arcola?adjus...,38,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",2.0,[],"{'address1': '5243 Fm 521 Rd', 'address2': 'St...",1.281432e+10,(281) 431-9464,3479.181431,NaN,29.507534,-95.459899,Error
3,b4a8eGt5rBr8ubKGXO8wYA,take-the-wheel-mobile-fresno,Take The Wheel Mobile,https://s3-media2.fl.yelpcdn.com/bphoto/WWd4yD...,False,https://www.yelp.com/biz/take-the-wheel-mobile...,59,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"['pickup', 'delivery']","{'address1': '4233 Fm 521 Rd', 'address2': Non...",1.832334e+10,(832) 334-2181,2171.094342,$,29.523673,-95.454140,Error
4,tsLFO0SuZPjqCtGcDNnWig,bar-kada-missouri-city,Bar Kada,https://s3-media3.fl.yelpcdn.com/bphoto/WEs1ij...,False,https://www.yelp.com/biz/bar-kada-missouri-cit...,49,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",3.5,[],"{'address1': '9009 Sienna Crossing Dr', 'addre...",1.713228e+10,(713) 227-5232,6197.980513,NaN,29.537163,-95.533925,Error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Pql6fHY89HmU2xYit08FIw,sonic-drive-in-pearland-7,Sonic Drive-In,https://s3-media4.fl.yelpcdn.com/bphoto/3sL9Hk...,False,https://www.yelp.com/biz/sonic-drive-in-pearla...,79,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"['delivery', 'pickup']","{'address1': '11309 Shadow Creek Pkwy', 'addre...",1.713340e+10,(713) 340-0806,8660.337247,$,29.580023,-95.394722,Error
197,bAgiCCzQUO4QyD26sX4YUw,silverlake-hunan-restaurant-pearland-2,Silverlake Hunan Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/o1HT5H...,False,https://www.yelp.com/biz/silverlake-hunan-rest...,100,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.0,['delivery'],"{'address1': '10223 Broadway St', 'address2': ...",1.713437e+10,(713) 436-9988,8971.510555,$,29.556560,-95.380520,Error
198,xPUTiVph-0PTYL-NAxAvGA,old-hickory-inn-missouri-city,Old Hickory Inn,https://s3-media1.fl.yelpcdn.com/bphoto/rVMKy_...,False,https://www.yelp.com/biz/old-hickory-inn-misso...,82,"[{'alias': 'bbq', 'title': 'Barbeque'}]",3.5,"['delivery', 'pickup']","{'address1': '3334 Fm 1092 Rd', 'address2': 'S...",1.281500e+10,(281) 499-8904,9891.253375,$$,29.571845,-95.564201,Error
199,D7bjSkrIekLMAv-PB83bGQ,chuck-e-cheese-pearland-2,Chuck E. Cheese,https://s3-media2.fl.yelpcdn.com/bphoto/P_Evbf...,False,https://www.yelp.com/biz/chuck-e-cheese-pearla...,52,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"['delivery', 'pickup']","{'address1': '3141 Silverlake Village Dr', 'ad...",1.713340e+10,(713) 340-1530,8277.198159,$,29.551846,-95.385833,Error


In [37]:
## slice out a new test address and inspect
test_addr = df.loc[100, 'location']
test_addr

'{"address1": "14608 S Post Oak Rd", "address2": "", "address3": "", "city": "Houston", "zip_code": "77045", "country": "US", "state": "TX", "display_address": ["14608 S Post Oak Rd", "Houston, TX 77045"]}'

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with " ' " in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [38]:
## remove any rows where display_location == 'ERROR'
df = df[df['display_location'] != 'Error']
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location
7,HjHjZnRqXYE5_jRwV4pPrQ,bonchon-pearland-6,Bonchon,https://s3-media4.fl.yelpcdn.com/bphoto/_JgCWc...,False,https://www.yelp.com/biz/bonchon-pearland-6?ad...,303,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"['pickup', 'delivery']","{""address1"": ""11200 Broadway St"", ""address2"": ...",1.346411e+10,(346) 410-5167,7497.781280,$$,29.552888,-95.394326,"{'address1': '11200 Broadway St', 'address2': ..."
8,xkqqY29BviSEtSZeFzc2cg,big-mammas-home-cooking-missouri-city,Big Mammas Home Cooking,https://s3-media4.fl.yelpcdn.com/bphoto/Cywt_e...,False,https://www.yelp.com/biz/big-mammas-home-cooki...,54,"[{'alias': 'soulfood', 'title': 'Soul Food'}, ...",3.5,"['pickup', 'delivery']","{""address1"": ""8731 Hwy 6"", ""address2"": ""Ste 10...",1.832440e+10,(832) 440-2984,6660.500363,NaN,29.540766,-95.538571,"{'address1': '8731 Hwy 6', 'address2': 'Ste 10..."
9,ZaISh61fxTQE11qv4vvN7w,wingstop-missouri-city-2,Wingstop,https://s3-media4.fl.yelpcdn.com/bphoto/_rLChq...,False,https://www.yelp.com/biz/wingstop-missouri-cit...,58,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",2.5,[],"{""address1"": ""7260 Highway 6"", ""address2"": ""St...",1.281261e+10,(281) 261-2300,8325.799321,$$,29.552562,-95.554152,"{'address1': '7260 Highway 6', 'address2': 'St..."
11,pTKZxpNrHL7hFyu54PGbig,micheauxs-southern-cuisine-missouri-city,Micheaux's Southern Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/Yb6V00...,False,https://www.yelp.com/biz/micheauxs-southern-cu...,272,"[{'alias': 'soulfood', 'title': 'Soul Food'}, ...",4.5,"['pickup', 'delivery']","{""address1"": ""6850 Highway 6"", ""address2"": ""st...",1.832987e+10,(832) 987-1916,8670.870406,$$,29.556212,-95.556866,"{'address1': '6850 Highway 6', 'address2': 'st..."
13,-nB7nouFDL-2TWE5mNswwg,center-court-pizza-and-brew-pearland,Center Court Pizza & Brew,https://s3-media3.fl.yelpcdn.com/bphoto/pvRDJf...,False,https://www.yelp.com/biz/center-court-pizza-an...,282,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']","{""address1"": ""9721 Broadway St"", ""address2"": ""...",1.713436e+10,(713) 436-3927,9805.719621,$$,29.557210,-95.370956,"{'address1': '9721 Broadway St', 'address2': '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,2fLNuRe2-uhGuCTmMqYU-g,dennys-missouri-city,Denny's,https://s3-media4.fl.yelpcdn.com/bphoto/zgDfR-...,False,https://www.yelp.com/biz/dennys-missouri-city?...,91,"[{'alias': 'tradamerican', 'title': 'American ...",2.0,"['delivery', 'pickup']","{""address1"": ""6131 Highway 6 S"", ""address2"": ""...",1.281208e+10,(281) 208-2231,9819.727999,$,29.567578,-95.565296,"{'address1': '6131 Highway 6 S', 'address2': '..."
195,vJgd4vnYe3MMCFaB830fQg,baytown-seafood-restaurant-missouri-city,Baytown Seafood Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/Cd91Yz...,False,https://www.yelp.com/biz/baytown-seafood-resta...,75,"[{'alias': 'seafood', 'title': 'Seafood'}]",3.0,['delivery'],"{""address1"": ""2409 Fm 1092 Rd"", ""address2"": """"...",1.281499e+10,(281) 499-0622,10463.838028,$$,29.584227,-95.564078,"{'address1': '2409 Fm 1092 Rd', 'address2': ''..."
198,xPUTiVph-0PTYL-NAxAvGA,old-hickory-inn-missouri-city,Old Hickory Inn,https://s3-media1.fl.yelpcdn.com/bphoto/rVMKy_...,False,https://www.yelp.com/biz/old-hickory-inn-misso...,82,"[{'alias': 'bbq', 'title': 'Barbeque'}]",3.5,"['delivery', 'pickup']","{""address1"": ""3334 Fm 1092 Rd"", ""address2"": ""S...",1.281500e+10,(281) 499-8904,9891.253375,$$,29.571845,-95.564201,"{'address1': '3334 Fm 1092 Rd', 'address2': 'S..."
199,D7bjSkrIekLMAv-PB83bGQ,chuck-e-cheese-pearland-2,Chuck E. Cheese,https://s3-media2.fl.yelpcdn.com/bphoto/P_Evbf...,False,https://www.yelp.com/biz/chuck-e-cheese-pearla...,52,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"['delivery', 'pickup']","{""address1"": ""3141 

- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [ ]:
## slice out a new test address and inspect


In [39]:
## use apply and lambda to slice correct key
df['display_address'] = df['display_location'].apply(lambda x: x['display_address'])

C:\Users\jonnu\AppData\Local\Temp\ipykernel_20492\1245523748.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location,display_address
7,HjHjZnRqXYE5_jRwV4pPrQ,bonchon-pearland-6,Bonchon,https://s3-media4.fl.yelpcdn.com/bphoto/_JgCWc...,False,https://www.yelp.com/biz/bonchon-pearland-6?ad...,303,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"['pickup', 'delivery']","{""address1"": ""11200 Broadway St"", ""address2"": ...",1.346411e+10,(346) 410-5167,7497.781280,$$,29.552888,-95.394326,"{'address1': '11200 Broadway St', 'address2': ...","[11200 Broadway St, Ste 360, Pearland, TX 77584]"
8,xkqqY29BviSEtSZeFzc2cg,big-mammas-home-cooking-missouri-city,Big Mammas Home Cooking,https://s3-media4.fl.yelpcdn.com/bphoto/Cywt_e...,False,https://www.yelp.com/biz/big-mammas-home-cooki...,54,"[{'alias': 'soulfood', 'title': 'Soul Food'}, ...",3.5,"['pickup', 'delivery']","{""address1"": ""8731 Hwy 6"", ""address2"": ""Ste 10...",1.832440e+10,(832) 440-2984,6660.500363,NaN,29.540766,-95.538571,"{'address1': '8731 Hwy 6', 'address2': 'Ste 10...","[8731 Hwy 6, Ste 100, Missouri City, TX 77459]"


- Almost done! We want to convert display_address to a string instead of a list of strings.
- We can use the string method .join to do so!

In [41]:
## slice out a test_address
test_add = df.loc[7, 'display_address']
test_add

['11200 Broadway St', 'Ste 360', 'Pearland, TX 77584']

In [42]:
## test using .join with a "\n"
print('\n'.join(test_add))

11200 Broadway St
Ste 360
Pearland, TX 77584


In [43]:
## apply the join to every row with a lambda
df['display_address'] = df['display_address'].apply(lambda x: '\n'.join(x))

C:\Users\jonnu\AppData\Local\Temp\ipykernel_20492\1273990911.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location,display_address
7,HjHjZnRqXYE5_jRwV4pPrQ,bonchon-pearland-6,Bonchon,https://s3-media4.fl.yelpcdn.com/bphoto/_JgCWc...,False,https://www.yelp.com/biz/bonchon-pearland-6?ad...,303,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"['pickup', 'delivery']","{""address1"": ""11200 Broadway St"", ""address2"": ...",1.346411e+10,(346) 410-5167,7497.781280,$$,29.552888,-95.394326,"{'address1': '11200 Broadway St', 'address2': ...","11200 Broadway St\nSte 360\nPearland, TX 77584"
8,xkqqY29BviSEtSZeFzc2cg,big-mammas-home-cooking-missouri-city,Big Mammas Home Cooking,https://s3-media4.fl.yelpcdn.com/bphoto/Cywt_e...,False,https://www.yelp.com/biz/big-mammas-home-cooki...,54,"[{'alias': 'soulfood', 'title': 'Soul Food'}, ...",3.5,"['pickup', 'delivery']","{""address1"": ""8731 Hwy 6"", ""address2"": ""Ste 10...",1.832440e+10,(832) 440-2984,6660.500363,NaN,29.540766,-95.538571,"{'address1': '8731 Hwy 6', 'address2': 'Ste 10...","8731 Hwy 6\nSte 100\nMissouri City, TX 77459"


### Lastly, Fixing Transactions Column

In [45]:
# Looking at a test transaction
test_tran = df.loc[7, 'transactions']
test_tran

"['pickup', 'delivery']"

- This is also a string and needs to be converted to a list.

In [46]:
# Replacing single ' with double "
saved_test = test_tran.replace("'", '"')
saved_test

'["pickup", "delivery"]'

In [ ]:
# Using json.loads on saved_test

In [47]:
# Applying transformations to entire column
df['tran_split'] = df['transactions'].str.replace("'", '"')
# Create a new column where the single quotes are replaced with double quotes

# Apply json.loads to entire column
df['tran_split'] = df['tran_split'].apply(json.loads)

C:\Users\jonnu\AppData\Local\Temp\ipykernel_20492\1570801179.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jonnu\AppData\Local\Temp\ipykernel_20492\1570801179.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
type(df.loc[7, 'tran_split'])

list

In [52]:
# Converting transactions column into a one-hot-encoded column
exploded = df.explode('tran_split')
exploded[['name', 'transactions', 'tran_split']].head()

,name,transactions,tran_split
7,Bonchon,"['pickup', 'delivery']",pickup
7,Bonchon,"['pickup', 'delivery']",delivery
8,Big Mammas Home Cooking,"['pickup', 'delivery']",pickup
8,Big Mammas Home Cooking,"['pickup', 'delivery']",delivery
9,Wingstop,[],NaN


In [54]:
# remove NaNs and find unique values
col_to_make = exploded['tran_split'].dropna().unique()
col_to_make

array(['pickup', 'delivery'], dtype=object)

In [55]:
# Using a for loop with .str.contains to create new columns

for col in col_to_make:
    df[col] = df['transactions'].str.contains(col)

C:\Users\jonnu\AppData\Local\Temp\ipykernel_20492\2172904382.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jonnu\AppData\Local\Temp\ipykernel_20492\2172904382.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,display_phone,distance,price,latitude,longitude,display_location,display_address,tran_split,pickup,delivery
7,HjHjZnRqXYE5_jRwV4pPrQ,bonchon-pearland-6,Bonchon,https://s3-media4.fl.yelpcdn.com/bphoto/_JgCWc...,False,https://www.yelp.com/biz/bonchon-pearland-6?ad...,303,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"['pickup', 'delivery']",...,(346) 410-5167,7497.781280,$$,29.552888,-95.394326,"{'address1': '11200 Broadway St', 'address2': ...","11200 Broadway St\nSte 360\nPearland, TX 77584","[pickup, delivery]",True,True
8,xkqqY29BviSEtSZeFzc2cg,big-mammas-home-cooking-missouri-city,Big Mammas Home Cooking,https://s3-media4.fl.yelpcdn.com/bphoto/Cywt_e...,False,https://www.yelp.com/biz/big-mammas-home-cooki...,54,"[{'alias': 'soulfood', 'title': 'Soul Food'}, ...",3.5,"['pickup', 'delivery']",...,(832) 440-2984,6660.500363,NaN,29.540766,-95.538571,"{'address1': '8731 Hwy 6', 'address2': 'Ste 10...","8731 Hwy 6\nSte 100\nMissouri City, TX 77459","[pickup, delivery]",True,True


### Final Map

In [62]:
## make ourn final map and save as varaible
pfig = px.scatter_mapbox(df, lat='latitude', lon='longitude', mapbox_style='open-street-map',
                 hover_name='name', hover_data=['price', 'rating', 'pickup', 'delivery', 'display_address'],
                 color='rating')
pfig.show()

### Saving Final Figure

In [ ]:
## use fig.write_html to save map
